**Project - I (Review Project Analysis)**

**DESCRIPTION**:Help a leading mobile brand understand the voice of the customer by analyzing the reviews of their product on Amazon and the topics that customers are talking about. You will perform topic modeling on specific parts of speech. You’ll finally interpret the emerging topics.

**Problem Statement**: A popular mobile phone brand, Lenovo has launched their budget smartphone in the Indian market. The client wants to understand the VOC (voice of the customer) on the product. This will be useful to not just evaluate the current product, but to also get some direction for developing the product pipeline. The client is particularly interested in the different aspects that customers care about. Product reviews by customers on a leading e-commerce site should provide a good view.

**Domain**: Amazon reviews for a leading phone brand

**Analysis to be done**: POS tagging, topic modeling using LDA, and topic interpretation

**Content:** 

Dataset: ‘K8 Reviews v0.2.csv’

**Columns:**

* Sentiment: The sentiment against the review (4,5 star reviews are positive, 1,2 are negative)

* Reviews: The main text of the review

**Steps to perform:**

Discover the topics in the reviews and present it to business in a consumable format. Employ techniques in syntactic processing and topic modeling.

Perform specific cleanup, POS tagging, and restricting to relevant POS tags, then, perform topic modeling using LDA. Finally, give business-friendly names to the topics and make a table for business.

**Tasks: **

1. Read the .csv file using Pandas. Take a look at the top few records.

2. Normalize casings for the review text and extract the text into a list for easier manipulation.

3. Tokenize the reviews using NLTKs word_tokenize function.

4. Perform parts-of-speech tagging on each sentence using the NLTK POS tagger.

5. For the topic model, we should  want to include only nouns.

A. Find out all the POS tags that correspond to nouns.

B. Limit the data to only terms with these tags.

6. Lemmatize.

A. Different forms of the terms need to be treated as one.
 
B. No need to provide POS tag to lemmatizer for now.

7. Remove stopwords and punctuation (if there are any). 

8. Create a topic model using LDA on the cleaned-up data with 12 topics.
 
A. Print out the top terms for each topic.

B. What is the coherence of the model with the c_v metric?

9. Analyze the topics through the business lens.

A. Determine which of the topics can be combined.

10. Create topic model using LDA with what you think is the optimal number of topics

A. What is the coherence of the model?

11. The business should  be able to interpret the topics.

A. Name each of the identified topics.

B. Create a table with the topic name and the top 10 terms in each to present to the  business.

**Task-1: Read the .csv file using Pandas. Take a look at the top few records.**

In [ ]:
import pandas as pd
import time
start_time=time.time()


df=pd.read_csv("../input/review-project-analysis/K8 Reviews v0.2.csv")


In [ ]:
df.head()

**Task-2. Normalize casings for the review text and extract the text into a list for easier manipulation.**

In [ ]:
review=df['review'].str.lower().to_list()

In [ ]:
review[:5]

**Lets remove special characters and numbers otherwise these characters may spoil text preprocessing.**

In [ ]:
import re

review = [re.sub("[.,|:='~^0-9\\\]", " ", item) for item in review]
              

**Task-3. Tokenize the reviews using NLTKs word_tokenize function.**

In [ ]:
from nltk import word_tokenize
review_tokens=[word_tokenize(item) for item in review]
print(review_tokens[:5])

**Task-4. Perform parts-of-speech tagging on each sentence using the NLTK POS tagger.**

In [ ]:
from nltk import pos_tag

review_postage=[pos_tag(item) for item in review_tokens]
print(review_postage[:5])

**Task-5. For the topic model, we should want to include only nouns.**

1.Find out all the POS tags that correspond to nouns.

2.Limit the data to only terms with these tags

In [ ]:
import nltk
nltk.help.upenn_tagset()


In [ ]:
noun_tags = ['NN', 'NNS', 'NNP', 'NNPS']

review_postags_nouns = []

for item in review_postage:
    noun_tokens = [token_tag[0] for token_tag in item if token_tag[1] in noun_tags]
    review_postags_nouns.append(noun_tokens)
    
print(review_postags_nouns[:5])

**Task-6. Lemmatize**

1.Different forms of the terms need to be treated as one.

2.No need to provide POS tag to lemmatizer for now.

In [ ]:
from nltk import WordNetLemmatizer

wnl = WordNetLemmatizer()

review_postags_nouns_lemmed = []

for item in review_postags_nouns:
    lemmed_tokens = [wnl.lemmatize(token, 'n') for token in item]
    review_postags_nouns_lemmed.append(lemmed_tokens)
    
print(review_postags_nouns_lemmed[:5])

**Task-7. Remove stopwords and punctuation (if there are any).**

In [ ]:
from nltk.corpus import stopwords
import string

sw = stopwords.words("english")
punc = list(string.punctuation)

custom_sw = sw + punc

review_preprocessed = []

for item in review_postags_nouns_lemmed:
    if len(item)>0:        
        preprocessed_tokens = [token for token in item if token not in custom_sw and len(token)>1]
        review_preprocessed.append(preprocessed_tokens)
    else:
        review_preprocessed.append(item)
    
print(review_preprocessed[:5])

**Task-8. Create a topic model using LDA on the cleaned-up data with 12 topics.**

1.Print out the top terms for each topic.

2.What is the coherence of the model with the c_v metric?

In [ ]:
import gensim
from gensim.corpora import Dictionary
import numpy as np

np.random.seed(100)

In [ ]:
dictionary = Dictionary(review_preprocessed)

In [ ]:
type(dictionary)

In [ ]:
dictionary.filter_extremes(no_below=5, no_above=.8 ,keep_n=None)

In [ ]:
print(dictionary)

In [ ]:
bow_text = [dictionary.doc2bow(item) for item in review_preprocessed]

In [ ]:
bow_text[:5]

In [ ]:
sample_bow = bow_text[4]

for item in sample_bow:
    print("Word '{}' comes {} times in this sample review".format(dictionary[item[0]], item[1]))

In [ ]:
lda_model=gensim.models.LdaMulticore(bow_text,num_topics=12,id2word=dictionary,random_state=1,passes=50)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print("\nTopic: {} \nWords: {}".format(idx, topic ))

In [ ]:
from gensim.models import CoherenceModel
coherence_score = CoherenceModel(model=lda_model, texts=review_preprocessed, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_score.get_coherence()
print('Coherence Score for this LDA model is: ', coherence_lda)

**Task-9. Analyze the topics through the business lens.**

1.Determine which of the topics can be combined.

In [ ]:
# Let us visualize topics in our text

import pyLDAvis
from pyLDAvis import gensim as gensim_vis
import warnings
warnings.simplefilter('ignore')

pyLDAvis.enable_notebook()

In [ ]:
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_text, dictionary)
LDAvis_prepared

In [ ]:
pyLDAvis.save_html(LDAvis_prepared, 'LDA_model_vis'+'.html')

**Task-10. Create topic model using LDA with what you think is the optimal number of topics**

1.What is the coherence of the model?

In [ ]:
# From our exercise in previous step, we find that optimum no. of topics will 6 for given text.
# Let us again create model with 6 topics

lda_model1 = gensim.models.LdaMulticore(bow_text, 
                                   num_topics = 6, 
                                   id2word = dictionary,  
                                   random_state=1,
                                   passes = 50)

for idx, topic in lda_model1.print_topics(-1):
    print("\nTopic: {} \nWords: {}".format(idx, topic ))

In [ ]:
coherence_score1 = CoherenceModel(model=lda_model1, texts=review_preprocessed, dictionary=dictionary, coherence='c_v')
coherence_lda1 = coherence_score1.get_coherence()
print('Coherence Score for new LDA model_1 is: ', coherence_lda1)

**Task-11. The business should be able to interpret the topics.**

1.Name each of the identified topics.

2.Create a table with the topic name and the top 10 terms in each to present to the business.

In [ ]:
topic_words = {}

for idx, topic in lda_model1.print_topics(-1): 
    temp = []
    for item in topic.split('+'):
        item_alpha = [letter for letter in item if letter.isalpha()]
        temp.append("".join(item_alpha))    
    topic_words[('Topic_'+str(idx+1))] = temp

topic_table = pd.DataFrame(topic_words)    
topic_table.index = ['Word_'+str(i+1) for i in range(topic_table.shape[0])]
topic_table

In [ ]:
x = lda_model1.show_topics(formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

for topic, words in topics_words:
    print(str(topic)+"::"+str(words))
    
print()

In [ ]:
LDAvis_prepared1 = pyLDAvis.gensim.prepare(lda_model1, bow_text, dictionary)
LDAvis_prepared1
pyLDAvis.save_html(LDAvis_prepared1, 'LDA_model_vis_1'+'.html')

In [ ]:
LDAvis_prepared1